<br><br><br> __<h1 style="text-align:center;"> Retail-Order and Sales Analysis with Python & MySQL  📊</h1>__
<h1 style="text-align:center;                                         "> Developed by Ramesh Sabhavath 👨🏼‍💻</h1> <br><br><br>

### import required Libraries

In [1]:

!pip install kaggle 
!pip install pandas sqlalchemy pyodbc
import kaggle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
import kaggle
print("Kaggle API is installed correctly!")

Kaggle API is installed correctly!


In [4]:
import sys, os
print(os.path.dirname(sys.executable))

C:\Anaconda Navigator


In [7]:
import os

# Tell Kaggle API to use current directory for kaggle.json

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()


In [2]:
      # download dataset using kaggle API 

!kaggle datasets download ankitbansal06/retail-orders -f orders.csv


  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 7.08MB/s]


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0



In [3]:
                                                      # extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()         # extract file to dir
zip_ref.close()              # close file

In [35]:
                                        # Read data from the file and handle null values

import pandas as pd

df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
df['Ship Mode'].unique()


array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

### Rename columns names and make them lower case and replace space with underscore


In [36]:

df.rename(columns={'Order Id':'order_id', 'City': 'city'}) 
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [37]:
df.shape

(9994, 16)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_id          9994 non-null   int64 
 1   order_date        9994 non-null   object
 2   ship_mode         9988 non-null   object
 3   segment           9994 non-null   object
 4   country           9994 non-null   object
 5   city              9994 non-null   object
 6   state             9994 non-null   object
 7   postal_code       9994 non-null   int64 
 8   region            9994 non-null   object
 9   category          9994 non-null   object
 10  sub_category      9994 non-null   object
 11  product_id        9994 non-null   object
 12  cost_price        9994 non-null   int64 
 13  list_price        9994 non-null   int64 
 14  quantity          9994 non-null   int64 
 15  discount_percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB


In [39]:
df.isnull().sum()

order_id            0
order_date          0
ship_mode           6
segment             0
country             0
city                0
state               0
postal_code         0
region              0
category            0
sub_category        0
product_id          0
cost_price          0
list_price          0
quantity            0
discount_percent    0
dtype: int64

In [40]:
df['ship_mode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9994 entries, 0 to 9993
Series name: ship_mode
Non-Null Count  Dtype 
--------------  ----- 
9988 non-null   object
dtypes: object(1)
memory usage: 78.2+ KB


In [41]:
df['ship_mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [43]:
df['ship_mode'].mode()

0    Standard Class
Name: ship_mode, dtype: object

In [ ]:
# replacing null values with the mode

In [44]:

df['ship_mode'] = df['ship_mode'].fillna('Standard Class')
df['ship_mode'] 


0         Second Class
1         Second Class
2         Second Class
3       Standard Class
4       Standard Class
             ...      
9989      Second Class
9990    Standard Class
9991    Standard Class
9992    Standard Class
9993      Second Class
Name: ship_mode, Length: 9994, dtype: object

In [45]:
df['ship_mode'] .info()

<class 'pandas.core.series.Series'>
RangeIndex: 9994 entries, 0 to 9993
Series name: ship_mode
Non-Null Count  Dtype 
--------------  ----- 
9994 non-null   object
dtypes: object(1)
memory usage: 78.2+ KB


In [47]:
df['ship_mode'].isnull().sum()

0

In [49]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,9994.0,4997.500000,2885.163629,1.0,2499.25,4997.5,7495.75,9994.0
postal_code,9994.0,55190.379428,32063.693350,1040.0,23223.00,56430.5,90008.00,99301.0
cost_price,9994.0,201.189714,537.743203,0.0,20.00,50.0,180.00,18110.0
list_price,9994.0,229.756854,623.245839,0.0,20.00,50.0,210.00,22640.0
quantity,9994.0,3.789574,2.225110,1.0,2.00,3.0,5.00,14.0
discount_percent,9994.0,3.484090,1.114211,2.0,2.00,3.0,4.00,5.0


## Usoing this clean Data for Analysing..

In [ ]:
# Drive new columns discount, sale price and profit

In [10]:

df['discount']=df['list_price']*df['discount_percent']*.01

df['sale_price']=df['list_price']-df['discount']

df['profit']=df['sale_price']-df['cost_price']

df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [ ]:
# convert order date from object data type to datetime

In [12]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [50]:
df['order_date']=pd.to_datetime(df['order_date'],format="mixed")

In [54]:
df['order_date'].describe()

count                             9994
mean     2022-12-29 10:20:34.820892672
min                2022-01-01 00:00:00
25%                2022-06-29 00:00:00
50%                2022-12-28 00:00:00
75%                2023-07-03 00:00:00
max                2023-12-31 00:00:00
Name: order_date, dtype: object

In [ ]:
# drop cost price, list price and discount percent columns

In [15]:
df.drop(['list_price', 'cost_price', 'discount_percent'], axis=1, inplace=True)
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

 # Connect to MYSQL

In [30]:

!pip install mysql-connector-python pandas ipython-sql
!pip install sqlalchemy pymysql pandas ipython-sql
!pip install mysql-connector-python sqlalchemy pymysql pandas
!pip install mysql-connector-python pandas



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [19]:

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine


In [31]:
file_path = r"C:\Users\woake\Python\orders.csv"  # adjust if needed
df = pd.read_csv(file_path)
df.head() 

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [21]:
conn = mysql.connector.connect(
    host="localhost:3306",
    user="root",              # Usually 'root' for local MySQL; you said username is localhost? We'll use 'root'
    password="",                # Empty since no password
    database="orders_data"        # Replace with your database
)
cursor = conn.cursor()
for i, row in df.iterrows():
    sql = " select * from orders_table"
    cursor.execute(sql, tuple(row))
conn.commit()

print("CSV data successfully imported into MySQL!")

# Optional: Close the connection
# cursor.close()
# conn.close()


CSV data successfully imported into MySQL!


In [25]:
df.columns

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Order Id,9994.0,4997.500000,2885.163629,1.0,2499.25,4997.5,7495.75,9994.0
Postal Code,9994.0,55190.379428,32063.693350,1040.0,23223.00,56430.5,90008.00,99301.0
cost price,9994.0,201.189714,537.743203,0.0,20.00,50.0,180.00,18110.0
List Price,9994.0,229.756854,623.245839,0.0,20.00,50.0,210.00,22640.0
Quantity,9994.0,3.789574,2.225110,1.0,2.00,3.0,5.00,14.0
Discount Percent,9994.0,3.484090,1.114211,2.0,2.00,3.0,4.00,5.0


In [34]:
from sqlalchemy import create_engine
import pandas as pd

# I have not SET ANY PASSWORD TO MYSQL

engine = create_engine("mysql+pymysql://root:@localhost:3306/orders")

query = "SELECT * FROM orders_table LIMIT 10;"
df = pd.read_sql(query, engine)

print(df.head())

df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5
5,6,2022-03-13,Not Available,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,10,10,4,3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,860,910,6,5
8,9,2023-03-23,Not Available,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,20,20,3,2
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,90,110,5,3


In [60]:
# Load the data into mysql using append option

df.to_sql('df_orders', con=conn, index=False, if_exists = 'append')

38